In [1]:
import os

In [2]:
%pwd

'/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/khalidasdsju/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="khalidasdsju"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6acb7ed9b72b1a8cc4ca112fb46cc677e704b825"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

Entity

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

ConfigurationManager in src

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/khalidasdsju/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    import traceback
    print("An error occurred during evaluation pipeline:")
    traceback.print_exc()
    raise e


[2025-04-17 16:57:25,300: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-17 16:57:25,301: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-17 16:57:25,301: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.


/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/chest/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 831ms/step - accuracy: 0.9258 - loss: 0.6270
[2025-04-17 16:57:31,588: INFO: common: json file saved at: scores.json]
An error occurred during evaluation pipeline:


Traceback (most recent call last):
  File "/var/folders/vw/y91jspq93j137smxkxhblnmm0000gn/T/ipykernel_1870/3229007403.py", line 6, in <module>
    evaluation.log_into_mlflow()
  File "/var/folders/vw/y91jspq93j137smxkxhblnmm0000gn/T/ipykernel_1870/3217913975.py", line 51, in log_into_mlflow
    with mlflow.start_run():
  File "/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/chest/lib/python3.10/site-packages/mlflow/tracking/fluent.py", line 350, in start_run
    active_run_obj = client.create_run(
  File "/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/chest/lib/python3.10/site-packages/mlflow/tracking/client.py", line 275, in create_run
    return self._tracking_client.create_run(experiment_id, start_time, tags, run_name)
  File "/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/chest/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/client.py", line 131, in create_run
    return self.

RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [18]:
# Import necessary libraries
import mlflow
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
from urllib.parse import urlparse
from cnnClassifier.utils.common import read_yaml, create_directories, save_json
from datetime import datetime

# Step 1: Define configuration classes
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(self, config_filepath="config.yaml", params_filepath="params.yaml"):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/khalidasdsju/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


# Step 2: Define the Evaluation Class
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255, 
            validation_split=0.30  
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(Path(self.config.path_of_model))
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path(f"scores_{str(datetime.now())}.json"), data=scores)

    def log_into_mlflow(self):
        try:
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            print(f"Tracking URI set to: {self.config.mlflow_uri}")
            
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            print(f"Tracking URI scheme: {tracking_url_type_store}")
            
            experiments = mlflow.list_experiments()
            print(f"Available experiments: {experiments}")

            with mlflow.start_run():
                print(f"Starting MLflow run...")
                print(f"Logging parameters: {self.config.all_params}")
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )

                if tracking_url_type_store != "file":
                    print(f"Registering model to the remote MLflow registry...")
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    print(f"Logging model locally...")
                    mlflow.keras.log_model(self.model, "model")
                
        except Exception as e:
            print(f"Error occurred while logging to MLflow: {e}")
            raise e


# Step 3: Running the Full Evaluation Pipeline
try:
    config = ConfigurationManager()  
    eval_config = config.get_evaluation_config()  
    evaluation = Evaluation(eval_config)  
    evaluation.evaluation()  
    evaluation.log_into_mlflow()  

except Exception as e:
    import traceback
    print("An error occurred during evaluation pipeline:")
    traceback.print_exc()
    raise e  # Re-raise the error for further handling or logging





An error occurred during evaluation pipeline:


Traceback (most recent call last):
  File "/var/folders/vw/y91jspq93j137smxkxhblnmm0000gn/T/ipykernel_1870/1957161277.py", line 113, in <module>
    config = ConfigurationManager()
  File "/var/folders/vw/y91jspq93j137smxkxhblnmm0000gn/T/ipykernel_1870/1957161277.py", line 22, in __init__
    self.config = read_yaml(config_filepath)
  File "/Users/khalid/Desktop/End-to-End-Chest-Cancer-Classification-using-MLflow-DVC/chest/lib/python3.10/site-packages/ensure/main.py", line 845, in __call__
    raise EnsureError(msg.format(
ensure.main.EnsureError: Argument path_to_yaml of type <class 'str'> to <function read_yaml at 0x1053e0550> does not match annotation type <class 'pathlib.Path'>


EnsureError: Argument path_to_yaml of type <class 'str'> to <function read_yaml at 0x1053e0550> does not match annotation type <class 'pathlib.Path'>